## FOLIUM:

Folium is a powerful Python library that helps you create several types of Leaflet maps. By default, Folium creates a map in a separate HTML file.

#### Adventages
Folium makes easy to visualize data that’s been manipulated in Python on an interactive leaflet map. It enables both the binding of data to a map for choropleth visualizations as well as passing rich vector/raster/HTML visualizations as markers on the map.

The library has a number of built-in tilesets from OpenStreetMap, Mapbox, and Stamen, and supports custom tilesets with Mapbox or Cloudmade API keys. folium supports both Image, Video, GeoJSON and TopoJSON overlays.

#### Starting point

Folium.Map(location = [LAT0,LON0], zoom_start = 12)

location has a specific coordinates and zoom shows the distance from the coordinate.


In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString


import branca as br 

In [ ]:
#Gettting the character format

base = open(r'../_data/Folium/MINING.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']

In [ ]:
# Geographical information of mining

MINING = pd.read_csv( r'../_data/Folium/MINING.csv', encoding = charenc)
MINING

In [ ]:
# Mining activities in Yauli - Junín

MINING1 = MINING[MINING.DISTRITO == "YAULI"]
MINING1 

In [ ]:
# Loop to save coordinates from mining location

for j in range(0,len(MINING1)):
    globals()[f'LON{j}'] = MINING1.iloc[j,8]
    globals()[f'LAT{j}'] = MINING1.iloc[j,9]

## 1. First Map and some additional details

In [ ]:
# Creating the Map 

a = fm.Map(location = [LAT0,LON0], tiles="Stamen Terrain", zoom_start = 12, control_scale=True)

tooltip = "Click"

#Mineral type in a list 

mineral = ["CASAPALCA (Zinc)","ARGENTUM (Cobre)","VOLCAN (Mercurio)", "ARGENTUM (Plomo)", "VOLCAN (Plata)", "CHINALCO (Caliza)"]

# Adding circle and mark. 
# popup save information 

for i in range(0,len(MINING1)):
    fm.CircleMarker([globals()[f'LAT{i}'], globals()[f'LON{i}']], popup= mineral[i],
              radius = 100,
              fill=True,
              fill_color="#3186cc",
              tooltip=tooltip).add_to(a)

    for i in range(0,len(MINING1)):
        fm.Marker([globals()[f'LAT{i}'], globals()[f'LON{i}']], popup= mineral[i],
              radius = 50,
              icon=fm.Icon(color="red", icon="info-sign"),
              color="#3186cc",  
              fill=True,
              fill_color="#3186cc",
              tooltip=tooltip).add_to(a)
a

In [ ]:
# Alternative Tile

m = fm.Map(location = [LAT0,LON0], tiles="cartodbpositron", zoom_start = 13)
m

In [ ]:
# Uplaoad economic indicators such us IDH, GDP per - capita, Poverty rate 

base = open(r'../_data/Folium/Poverty.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']

poverty = pd.read_csv( r'../_data/Folium/Poverty.csv', encoding = charenc)
poverty

In [ ]:
#Check variables´ types to make math operations, merge datasets, etc
poverty.dtypes

In [ ]:
# Yoy must upload coordinates base on GeoJson format to use package Choropleth. 

distritos = gpd.read_file(r'../_data/folium/peru_distrital_simple.geojson')
distritos1 = distritos[['IDDIST', 'geometry']]
distritos1 = distritos1 .rename({'IDDIST':'UBIGEO1'}, axis =1 )
distritos1['UBIGEO1'] = distritos1['UBIGEO1'].astype(str).astype(np.int64)

In [ ]:
#administrative regions 

dpto = gpd.read_file(r'../_data/folium/peru_departamental_simple.geojson')
dpto1 = dpto[['FIRST_IDDP', 'geometry']]
dpto1 = dpto1.rename({'FIRST_IDDP':'UBIGEO2'}, axis =1 )
dpto1['UBIGEO2'] = dpto1['UBIGEO2'] + "0000"
dpto1['UBIGEO2'] = dpto1['UBIGEO2'].astype(str).astype(np.int64)
dpto1

In [ ]:
# From district level to administrative region

poverty2 = poverty.drop_duplicates(subset=['UBIGEO2'])
poverty2 = poverty2.drop(689)
poverty2['IDH'] = poverty2['IDH'].astype(str).astype(float)

In [ ]:
poverty2['IDH'] = poverty2['IDH'].astype(str).astype(float)
poverty2['PBI_PC'] = poverty2['PBI_PC'].astype(str).astype(float)
poverty2

## 2. Choropleth 

In [ ]:
# government palace coordinates

lat_palacio = -12.0757538
long_palacio = -76.9863174

z = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = 5)

# Mandatory: geo_data in GeoJson format
# columns: variables from economics indicators data set
# Atention !!! key_on: commom variable between geodata and data "feature.properties.(name of variable)"


fm.Choropleth(
    geo_data=distritos1,
    data=poverty,
    columns=['UBIGEO1', 'POVERTY_RATE'],
    key_on="feature.properties.UBIGEO1",
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name="Poverty Rate (%)",
    smooth_factor=0,
    Highlight= True,
    line_color = "#0000",
    overlay=True,
    nan_fill_color = "White"  # fill white missing values 
    ).add_to(z)

fm.LayerControl().add_to(z)

# Save in a html format 

z.save("Poverty_Map.html")

z

In [ ]:
lat_palacio = -12.0757538
long_palacio = -76.9863174

# Add quantile on legend 

bins = list(poverty2["IDH"].quantile([0, 0.2, 0.4, 0.6,0.8, 1]))
z = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 5)

fm.Choropleth(geo_data=dpto1,
            data=poverty2,
            columns=["UBIGEO2", "IDH"],
            key_on="feature.properties.UBIGEO2",
            fill_color="Reds",  
            fill_opacity=0.8,
            legend_name="Human development index",
            bins = bins,
            reset = True
            ).add_to(z)

fm.LayerControl().add_to(z)

z

In [ ]:
lat_palacio = -12.0757538
long_palacio = -76.9863174


#add quantils

bins = list(poverty2["IDH"].quantile([0, 0.2, 0.4, 0.6,0.8, 1]))

z = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 5)

fm.Choropleth(geo_data=dpto1,
            data=poverty2,
            columns=["UBIGEO2", "IDH"],
            key_on="feature.properties.UBIGEO2",
            fill_color="Reds",  
            fill_opacity=0.8,
            legend_name="Human development index",
            bins = bins,
            reset = True
            ).add_to(z)

#Merge both dataset to add informaction by region 
data_both = pd.merge(dpto1, poverty2, how="inner", on="UBIGEO2")
data_both = data_both.to_json()    # from pandas to Json 

# Color and opacity of each region 
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

# Color and opacity of each selected region 

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.5, 
                                'weight': 0.1}

details = fm.features.GeoJson(
    data = data_both,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=fm.features.GeoJsonTooltip(
        fields=['DEPARTAMENTO','IDH'],   #Variable selection
        aliases=['Administrative Region', 'Human Development Index'],  # renames
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

# Add new features 

z.add_child(details)
z.keep_in_front(details)

z

In [ ]:
#Gettting the character format

base = open(r'../_data/Folium/enaho.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']

ENAHO = pd.read_csv( r'../_data/Folium/enaho.csv', encoding = charenc)
ENAHO

## 3. Heat Map and Interactive Map

In [ ]:


# beneficiary households of the universal bond 
bono_uni= ENAHO[ENAHO.bono_uni == 1]

# Coordinamtes in a list 

hogares = list(zip(bono_uni['latitud'], bono_uni['longitud']))

# List of tiles 

Tiles = ["stamenterrain","stamenwatercolor","cartodbpositron","openstreetmap", "cartodbdark_matter"]

z = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 12)


# Heat Map 
HeatMap(data=bono_uni[['latitud', 'longitud']], radius=20, name = 'Heatmap').add_to(z)
# Cluster Map
MarkerCluster(hogares, name = 'Cluster').add_to(z)

#Add different kind of tiles 

for i in Tiles:
    fm.TileLayer(i, name = i, control = True).add_to(z)
    
fm.LayerControl(collapsed=False).add_to(z)
z

## 4. Marker

In [ ]:
bonogas = ENAHO[ENAHO.bonogas == 1]

bn = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 12, control_scale=True)

# Loop for rows

for idx, row in bonogas.iterrows():
    Marker([row['latitud'], row['longitud']]).add_to(bn)

bn

## 5. Circles

In [ ]:
#unsatisfied basic need

#nbi1: Inadequate housing
#nbi2: Overcrowded housing
#nbi3: Housing without SSHH
#nbi4: School non-attendance
#nbi5: High economic dependency

base1 = ENAHO[ENAHO['nbi1'] == 1]
base2 = ENAHO[ENAHO['nbi2'] == 1]
base3 = ENAHO[ENAHO['nbi3'] == 1]
base4 = ENAHO[ENAHO['nbi4'] == 1]
base5 = ENAHO[ENAHO['nbi5'] == 1]

m = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 12.5, control_scale=True)

colors = ['forestgreen','darkred', 'blue', 'lime']

for j in range(1,5):
        
    for idx, row in globals()[f'base{j}'].iterrows():
        fm.Circle([row['latitud'], row['longitud']], radius = 200, color = colors[j-1]).add_to(m)

m

In [ ]:
# Marks and unsatisfied basic need


m = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 11, control_scale=True)

colors = ['purple','lightgreen', 'red', 'darkblue', 'orange']
nbi = ['Vivienda inadecuada','Vivienda con hacinamiento', 'Vivienda sin SS.HH', 'Inasistencia escolar', 'Alta depenencia Económica']

for j in range(1,5):
        
    for idx, row in globals()[f'base{j}'].iterrows():
         Marker([row['latitud'], row['longitud']], icon=fm.Icon(color=colors[j-1]), popup= nbi[j-1]).add_to(m)

        
m
    

## 6. Interactive Map part II

In [2]:
# Solidaridad Hospital Centers 

base = open(r'../_data/Folium/Solidaridad_Center.csv', 'rb').read()
det = chardet.detect(base)
charenc = det['encoding']

h_solidaridad = pd.read_csv( r'../_data/Folium/Solidaridad_Center.csv', encoding = charenc)
h_solidaridad

,Health_center,distrito,direction,Schedule,phone,especialidades,available_beds,Prueba_Covid,Centro_vacunacion,latitud,longitud
0,SISOL001,ATE,Av. Matropolitana 187,7:00-19:00,(01) 352-0798,35,15,No,Si,-12.033665,-76.942156
1,SISOL002,Santa Anita,Av. Cultura 808,7:00 - 20:00,(01) 231 3162,30,5,Si,Si,-12.046247,-76.968734
2,SISOL003,Agustino,Av. Cesar Vallejo con J.C Mariátegui,7:00 - 20:0,(01) 385 0954,35,20,Si,No,-12.039829,-76.997270
3,SISOL004,La Victoria,c. Manco Capac 218,8:00-18:00,(01) 426 4618,30,10,No,Si,-12.060868,-77.029709
4,SISOL005,Cercado de Lima,Jirón Camaná 7000,8:00-20:00,(01) 272 9422,28,11,Si,Si,-12.049331,-77.035446
5,SISOL006,Rimac,Av. Tupac Amaru Cdra.16,7:30-18:30,(01) 534 8556,20,25,No,No,-12.013246,-77.052052
6,SISOL007,Los Olivos,Av. Universitaria 15304,7:30-18:30,(01) 771 4269,30,15,Si,Si,-11.979238,-77.076962
7,SISOL008,Breña,Av. Colonial 2001,7:00-19:00,(01) 336 8228,25,20,Si,Si,-12.049191,-77.068206
8,SISOL009,Magdalena del Mar,Jr. Bolognesi 2013,7:00 - 20:00,(01) 263 6103,20,5,No,Si,-12.092302,-77.070088
9,SISOL010,Surquillo,Av. Angamos 734,8:00-18:00,(01) 243 1120,34,15,Si,Si,-12.113604,-77.023852


In [ ]:
# Function create table by each Health center using html. This funtion will be aplly by each row
# Almost alway each code on html requires a beginnig <p> and ending </p> 

def visual_html(i):
 
    # information by Health center 

    
    district = h_solidaridad['distrito'].iloc[i]                             
    direction = h_solidaridad['direction'].iloc[i]                           
    atencion = h_solidaridad['Schedule'].iloc[i]  
    phone = h_solidaridad['phone'].iloc[i]  
    espec = h_solidaridad['especialidades'].iloc[i]
    beds = h_solidaridad['available_beds'].iloc[i]
    covid = h_solidaridad['Prueba_Covid'].iloc[i]
    vacunation = h_solidaridad['Centro_vacunacion'].iloc[i]
    Health_center = h_solidaridad['Health_center'].iloc[i]
    
    #Color by each column of table 
    
    left_col_colour = "#FA8072"
    right_col_colour = "#BDC3C7"
    
    html = """<!DOCTYPE html>
<html>

<head>
    <p> Solidaridad Health Center </p>

</head>
    <table style="height: 126px; width: 350px;">  <!-- Comment: Create a teable. -->

<!-- Add information  -->

<tbody> 
<tr>

<!-- Add color by column -->

<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">District of Lima</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(district) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Direction</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(direction) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Openning Hour</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(atencion) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Phone - number</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(phone) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Number of medical specialties</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(espec) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Available beds</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(beds) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Covid test</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(covid) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Vaccination center</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(vacunation) + """
</tr>

</tbody>
</table>
</html>
"""
    return html

In [ ]:
ubication = h_solidaridad['latitud'].mean(), h_solidaridad['longitud'].mean()  # Average point

sol = fm.Map(location = ubication, zoom_start=12)

for i in range(0,len(h_solidaridad)):
    html = visual_html(i)

    iframe = br.element.IFrame(html=html,width=350,height=300)
    popup = fm.Popup(iframe,parse_html=True)
    
    fm.Marker([h_solidaridad['latitud'].iloc[i],h_solidaridad['longitud'].iloc[i]],
                  popup=popup,icon=fm.Icon(color= 'blue', icon='medkit', prefix="fa")).add_to(sol)

sol

In [3]:
# Alternative

def visual_html(i):
    
        html="""
        <h4>Direction: </h4>""" + str(h_solidaridad.iloc[i]['distrito']) + " - " + str(h_solidaridad.iloc[i]['direction']) +\
         """<h4>Phone - number:</h4>""" + str(h_solidaridad.iloc[i]['phone']) +\
        """<h4>Openning hour:</h4>""" + str(h_solidaridad.iloc[i]['Schedule']) +\
         """<h4>Number of medical specialties:</h4>""" + str(h_solidaridad.iloc[i]['especialidades']) +\
         """<h4>Available_beds:</h4>""" + str(h_solidaridad.iloc[i]['available_beds']) +\
        """<h4>Covid Test:</h4>""" + str(h_solidaridad.iloc[i]['Prueba_Covid']) +\
        """<h4>Vaccination center:</h4>""" + str(h_solidaridad.iloc[i]['Centro_vacunacion']) 
        return html


In [4]:
ubication = h_solidaridad['latitud'].mean(), h_solidaridad['longitud'].mean()

sol = fm.Map(location = ubication, zoom_start=12)

for i in range(0,len(h_solidaridad)):
    html = visual_html(i)

    iframe = br.element.IFrame(html=html,width=350,height=300)
    popup = fm.Popup(iframe,parse_html=True)
    
    fm.Marker([h_solidaridad['latitud'].iloc[i],h_solidaridad['longitud'].iloc[i]],
                  popup=popup,icon=fm.Icon(color= 'red', icon='medkit', prefix="fa")).add_to(sol)

sol.save("hospitals.html")

sol

### References:

##### MINEM Geografic mining centers 

http://www.minem.gob.pe/_publicaSector.php?idSector=1&idCategoria=24

##### Poverty map at distric level

https://www.inei.gob.pe/cifras-de-pobreza/

##### Geo-spatial information

https://visor.geoperu.gob.pe/

##### Folium 

https://python-visualization.github.io/folium/index.html

https://www.kaggle.com/alexisbcook/interactive-maps

https://www.kaggle.com/dabaker/fancy-folium

https://towardsdatascience.com/how-to-step-up-your-folium-choropleth-map-skills-17cf6de7c6fe

https://www.kaggle.com/mbnb8317/ds4c-tutorial-all-about-folium-pydeck